In [7]:
import pandas as pd

In [ ]:
%run application.py

 * Serving Flask app 'application' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://172.20.2.147:5000/ (Press CTRL+C to quit)


Loading BokehJS ...

172.20.2.147 - - [01/Jun/2022 05:06:48] "GET / HTTP/1.1" 200 -
172.20.2.147 - - [01/Jun/2022 05:06:48] "GET /static/css/bootstrap.min.css HTTP/1.1" 304 -
172.20.2.147 - - [01/Jun/2022 05:06:48] "GET /static/css/style.css HTTP/1.1" 304 -
172.20.2.147 - - [01/Jun/2022 05:06:48] "GET /static/figure/NYcity.jpg HTTP/1.1" 304 -
172.20.2.147 - - [01/Jun/2022 05:06:49] "GET /static/js/jquery.min.js HTTP/1.1" 304 -
172.20.2.147 - - [01/Jun/2022 05:06:49] "GET /static/js/bootstrap.min.js HTTP/1.1" 304 -
172.20.2.147 - - [01/Jun/2022 05:06:49] "GET /static/js/scripts.js HTTP/1.1" 304 -


In [2]:
df = pd.read_csv('./data/AB_NYC_2019.csv')

In [3]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [4]:
# string to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [5]:
# set NaN of last_review to earliest review time
earliest_time = df['last_review'].min()
df['last_review'][df['last_review'].isna()] = earliest_time

/tmp/ipykernel_2885/2655804918.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_review'][df['last_review'].isna()] = earliest_time


In [6]:
# add new col as timestamps of last_review time
df.insert(df.shape[1], 'last_review_ts', 0)
df['last_review_ts'] = df['last_review'].astype('int64')

In [7]:
# set NaN of reviews_per_month to 0
df['reviews_per_month'][df['reviews_per_month'].isna()] = 0

/tmp/ipykernel_2885/1341774583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviews_per_month'][df['reviews_per_month'].isna()] = 0


In [8]:
# turn text names to indices
neighbor_dict = {}
neighbor_group_dict = {}
room_type_dict = {}

In [9]:
for i, name in enumerate(df.neighbourhood.value_counts().index):
    neighbor_dict[name] = i
for i, name in enumerate(df.neighbourhood_group.value_counts().index):
    neighbor_group_dict[name] = i
for i, name in enumerate(df.room_type.value_counts().index):
    room_type_dict[name] = i

In [10]:
for row in df.iterrows():
    i, row = row[0], row[1]
    neighborhood = row['neighbourhood']
    df.loc[i, 'neighbourhood'] = neighbor_dict[row['neighbourhood']]
    df.loc[i, 'neighbourhood_group'] = neighbor_group_dict[row['neighbourhood_group']]
    df.loc[i, 'room_type'] = room_type_dict[row['room_type']]

In [11]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'last_review_ts'],
      dtype='object')

In [12]:
index = ['id', 'host_id', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'last_review_ts']

In [15]:
df_new = df[index]
df_new.to_csv('./data/data_preprocessed.csv', index=False)
pd.read_csv('./data/data_preprocessed.csv')

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,last_review_ts
0,2539,2787,1,52,40.64749,-73.97237,1,149,1,9,0.21,6,365,1539907200000000000
1,2595,2845,0,9,40.75362,-73.98377,0,225,1,45,0.38,2,355,1558396800000000000
2,3647,4632,0,2,40.80902,-73.94190,1,150,3,0,0.00,1,365,1301270400000000000
3,3831,4869,1,19,40.68514,-73.95976,0,89,1,270,4.64,1,194,1562284800000000000
4,5022,7192,0,10,40.79851,-73.94399,0,80,10,9,0.10,1,0,1542585600000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,8232441,1,1,40.67853,-73.94995,1,70,2,0,0.00,2,9,1301270400000000000
48891,36485057,6570630,1,3,40.70184,-73.93317,1,40,4,0,0.00,2,36,1301270400000000000
48892,36485431,23492952,0,2,40.81475,-73.94867,0,115,10,0,0.00,1,27,1301270400000000000
48893,36485609,30985759,0,5,40.75751,-73.99112,2,55,1,0,0.00,6,2,1301270400000000000
